In [10]:
#소리 파일을 다루기 위해 필요한 모듈
import numpy as np
import scipy.io.wavfile
import sounddevice as sd
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt

In [19]:
def open_wav(file_name, origin_rate):
    data, rate = librosa.load(file_name, sr=origin_rate)
    return data, rate

def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[start:end]

# 무음 구간을 찾아서 time_line을 반환한다.
# time_line은 리스트이고, 각 원소는 무음의 시작시간과 끝나는 시간을 튜플형태로 저장된다.
# 시간은 sample_rate의 형태로 저장된다. 시간으로 출력하기 위해서는 print_timeline을 이용한다.
def find_silence(data):
    time_count = 0
    flag = False
    time_line = []
    # silience_time은 2초간 침묵이 지속되면 삭제함을 의미한다.
    # threshold은 삭제될 소리의 크기의 역치이다. 1이면 모든 소리를 삭제히고 0이면 아무 소리도 삭제하지 않는다.
    # boundary는 자연스러운 소리를 위해 소리간의 여유분의 시간간격을 의미한다(0.5초) 
    silience_time = 192000*2
    threshold = 0.07
    boundary = int(192000/2)
    
    for i in range(0,len(data),1000):
        if abs(data[i]) < 0.1:
            if flag == False:
                flag = True
                start_index = i
            time_count += 1000
        elif time_count <= silience_time and abs(data[i]) >= threshold:
                time_count = 0
                flag = False
        if time_count >= silience_time and abs(data[i]) >= threshold:
            time_line.append((start_index+boundary,i-boundary))
            time_count = 0
            flag = False
    time_line.append((start_index+int(192000/2),len(data)))
    return time_line

# find_silence를 이용하여 time_line을 구한 후, data에서 무음 구간을 제거한 silence_removed를 반환한다.
def remove_silence(data,time_line):
    silence_removed = np.array([])
    sum = 0
    for i in range(len(time_line)-1):
        start = time_line[i][1]
        end = time_line[i+1][0]
        sum += end-start
        silence_removed = np.append(silence_removed,data[start:end])
    return silence_removed

# 디렉토리에 음성 파일들을 vocals + i의 형태로 가져다 둔 후, i의 범위를 넣으면 파일들을 자동으로 무음 구간을 제거해준다.
# 무음 구간이 제거된 파일은 result + i의 형태로 저장된다.
def Execute_Removing(start,end):
    print("Start Removing Silence from wav file..")
    for i in range(start,end+1):
        #read file data and prepare
        file_name = 'vocals'+str(i)+'.wav'
        sample_rate = 192000 # 192KHz
        data_length = sample_rate * 60 # 192KHz * 60
        wav_data, sample_rate = open_wav(file_name, sample_rate)
        data = np.array(wav_data)
        
        #run functions
        time_line = find_silence(data)
        silence_removed = remove_silence(data,time_line)
        
        #write results
        result_file_name = 'result'+str(i)+'.wav'
        sample_format = 'PCM_16' # PCM_24, PCM_16 PCM_s8 가능
        sf.write(result_file_name, silence_removed, sample_rate, subtype=sample_format)
    print("Completed!")
                             
def Execute_Removing_mp3(start,end):
    print("Start Removing Silence from wav file..")
    for i in range(start,end+1):
        #read file data and prepare
        file_name = 'vocals'+str(i)+'.mp3'
        sample_rate = 192000 # 192KHz
        data_length = sample_rate * 60 # 192KHz * 60
        wav_data, sample_rate = open_wav(file_name, sample_rate)
        data = np.array(wav_data)
        
        #run functions
        time_line = find_silence(data)
        silence_removed = remove_silence(data,time_line)
        
        #write results
        result_file_name = 'result'+str(i)+'.wav'
        sample_format = 'PCM_16' # PCM_24, PCM_16 PCM_s8 가능
        sf.write(result_file_name, silence_removed, sample_rate, subtype=sample_format)
    print("Completed!")
    
# time_line의 무음 구간을 시간의 형태로 출력한다.        
def print_timeline(time_line):
    for start,end in time_line:
        print(int(start/192000),int(end/192000))

# data를 그래프로 출력한다.
def show_sounds_in_graph(data):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(data)
    ax.set_xticks(list(range(0,wav_data.shape[0],int(wav_data.shape[0]/12))))
    ax.set_xticklabels(list(range(0,sec,int(sec/12))), fontsize=8)
    plt.show()

In [20]:
Execute_Removing(1,1)

Start Removing Silence from wav file..
Completed!
